# 0. IMPORTS

In [1]:
import inflection

import pandas     as pd
import numpy      as np
import seaborn    as sns
import scikitplot as skplt

from matplotlib        import pyplot          as plt
from matplotlib.pyplot import figure
from sklearn           import preprocessing   as pp
from sklearn           import model_selection as ms
from sklearn           import ensemble        as en
from sklearn           import neighbors       as nh
from sklearn           import linear_model    as lm

## 0.1. Loading Data

In [2]:
df_raw = pd.read_csv('dataset/train.csv')
df_raw.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


# 2. DATA DESCRIPTION

## 2.1. Data Preview

In [24]:
df_raw

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,16625,Male,55,28.0,26.0,1,> 2 Years,Yes,0,52413.0,16,1
1,16664,Female,45,28.0,26.0,1,1-2 Year,No,1,37968.0,57,0
2,16667,Female,72,8.0,124.0,1,1-2 Year,Yes,0,32433.0,89,0
3,16696,Female,46,28.0,124.0,1,1-2 Year,No,1,28805.0,145,0
4,16700,Female,69,33.0,122.0,1,1-2 Year,No,1,22214.0,156,0
5,16717,Male,54,28.0,26.0,1,1-2 Year,Yes,0,2630.0,93,1
6,16722,Female,69,28.0,122.0,1,1-2 Year,Yes,0,55051.0,114,0
7,16740,Female,48,28.0,26.0,1,1-2 Year,No,1,38845.0,133,0
8,16741,Female,51,3.0,124.0,1,1-2 Year,No,1,47538.0,297,0
9,16767,Male,46,8.0,124.0,1,1-2 Year,No,1,31948.0,98,0


## 2.2. Data Format, Types and Relevante Information

In [14]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gender                10 non-null     object 
 1   age                   10 non-null     int64  
 2   region_code           10 non-null     float64
 3   policy_sales_channel  10 non-null     float64
 4   driving_license       10 non-null     int64  
 5   vehicle_age           10 non-null     object 
 6   vehicle_damage        10 non-null     object 
 7   previously_insured    10 non-null     int64  
 8   annual_premium        10 non-null     float64
 9   vintage               10 non-null     int64  
 10  response              10 non-null     int64  
 11  id                    10 non-null     int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.1+ KB


In [17]:
np.round(df_raw.describe().T, 2)

,count,mean,std,min,25%,50%,75%,max
age,10.0,56.7,9.09,48.0,50.25,52.5,62.00,77.0
region_code,10.0,25.4,7.78,8.0,26.50,28.0,28.00,36.0
policy_sales_channel,10.0,45.6,43.27,1.0,26.00,26.0,47.75,124.0
driving_license,10.0,1.0,0.00,1.0,1.00,1.0,1.00,1.0
previously_insured,10.0,0.4,0.52,0.0,0.00,0.0,1.00,1.0
annual_premium,10.0,32240.2,16564.53,2630.0,30152.00,37410.5,43164.50,47660.0
vintage,10.0,142.0,74.84,46.0,91.50,136.0,199.25,263.0
response,10.0,0.2,0.42,0.0,0.00,0.0,0.00,1.0
id,10.0,19409.0,6810.26,27.0,21529.00,21544.5,21593.00,21623.0
